# Solution for the Titanic challenge using the Naïve-Bayes algorithm

**Libraries**

In [112]:
import pandas as pd
import numpy as np

**Data preprocessing**

In [113]:
titanic_train = pd.read_csv('/content/train.csv')
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [114]:
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [115]:
titanic_train.isnull().sum() # detecting null values

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [116]:
# removing null values and unecessary columns
titanic_train = titanic_train.dropna(subset=['Age'])
titanic_train = titanic_train.dropna(subset=['Embarked'])
titanic_train = titanic_train.drop(['Name', 'Cabin', 'Ticket' ], axis=1)

In [117]:
titanic_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
885,886,0,3,female,39.0,0,5,29.1250,Q
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [118]:
titanic_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

**Creating and treating the variables**

In [119]:
# independent variables
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
x_titanic_train = titanic_train[columns].values
x_titanic_train

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [1, 'female', 19.0, ..., 0, 30.0, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [120]:
# dependent variable
y_titanic_train = titanic_train['Survived'].values


In [121]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [122]:
# transforming nominal independent variables into numeric ones
one_hot_encoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 1, 6])], remainder='passthrough')
one_hot_encoder

ColumnTransformer(remainder='passthrough',
                  transformers=[('OneHot', OneHotEncoder(), [0, 1, 6])])

In [123]:
x_titanic_train_transformed = one_hot_encoder.fit_transform(x_titanic_train)

In [124]:
x_titanic_train_array = np.array(x_titanic_train_transformed)
x_titanic_train_array[0]

array([0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 22.0, 1, 0, 7.25],
      dtype=object)

**Aplying the Naïve-Bayes algorithm in the train data**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [141]:
# training the model
naive_titanic = GaussianNB()
naive_titanic.fit(x_titanic_train_array, y_titanic_train)

GaussianNB()

In [ ]:
# pclass, class, class, sex, sex, embarked, embarked, embarked, age, sibsp, parch, fare
prediction = naive_titanic.predict(x_titanic_train_array)
prediction

**Measuring the accuracy of the model**

In [150]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [145]:
accuracy_score(y_titanic_train, prediction)

0.776685393258427

In [148]:
confusion_matrix(y_titanic_train, prediction)

array([[344,  80],
       [ 79, 209]])

In [151]:
print(classification_report(y_titanic_train, prediction))

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       424
           1       0.72      0.73      0.72       288

    accuracy                           0.78       712
   macro avg       0.77      0.77      0.77       712
weighted avg       0.78      0.78      0.78       712



**Applying the model to test database**

In [199]:
titanic_test = pd.read_csv('/content/test.csv')

In [200]:
titanic_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [201]:
titanic_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [202]:
# Storing the id column
PassengerId = titanic_test['PassengerId'].values

In [203]:
# preparing the data
titanic_test = titanic_test.drop(['PassengerId','Name', 'Ticket','Cabin'], axis=1)
mean_age = titanic_test['Age'].mean()
titanic_test['Age'] = titanic_test['Age'].fillna(mean_age)
mean_fare = titanic_test['Fare'].mean()
titanic_test['Fare'] = titanic_test['Fare'].fillna(mean_fare)

In [204]:
titanic_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [205]:
test_array = titanic_test[:].values
test_array[0]

array([3, 'male', 34.5, 0, 0, 7.8292, 'Q'], dtype=object)

In [206]:
one_hot_encoder_test = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 1, 6])], remainder='passthrough')
titanic_test_transformed = one_hot_encoder.fit_transform(test_array)
titanic_test_transformed

array([[0.0, 0.0, 1.0, ..., 0, 0, 7.8292],
       [0.0, 0.0, 1.0, ..., 1, 0, 7.0],
       [0.0, 1.0, 0.0, ..., 0, 0, 9.6875],
       ...,
       [0.0, 0.0, 1.0, ..., 0, 0, 7.25],
       [0.0, 0.0, 1.0, ..., 0, 0, 8.05],
       [0.0, 0.0, 1.0, ..., 1, 1, 22.3583]], dtype=object)

**Generating the final results**

In [218]:
prediction_test = naive_titanic.predict(titanic_test_transformed)
prediction_test

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
titanic_result = {'PassengerId': PassengerId, 'Survived': prediction_test}
titanic_dataframe = pd.DataFrame(titanic_result )
titanic_dataframe

In [221]:
titanic_submission = titanic_dataframe.to_csv('titanic_submission.csv', index=False)